# スクレイピング

In [213]:
import requests
from bs4 import BeautifulSoup
import time
import sqlite3
import re  # 数字抽出用のライブラリ

# 初期設定
base_url = "https://www.jalan.net/theme/ski/"
query_base = "?roomCount=1&"
query_mid = "&adultNum=2&distCd=05&stayYear=&stayMonth=&dateUndecided=1&roomCrack=200000&vosFlg=6&mvTabFlg=1&stayDay=&screenId=UWW1402&activeSort=0&stayCount=1&rootCd=01126195&"

pref_codes = {
    "北海道": 126179,
    "青森": 126180,
    "岩手": 126181,
    "宮城": 126182,
    "秋田": 126183,
    "山形": 126184,
    "福島": 126185,
    "栃木": 126186,
    "群馬": 126187,
    "茨城": 126188,
    "埼玉": 126189,
    "千葉": 126190,
    "東京": 126191,
    "神奈川": 126192,
    "山梨": 126193,
    "長野": 126194,
    "新潟": 126195,
    "富山": 126196,
    "石川": 126197,
    "福井": 126198,
    "静岡": 126199,
    "岐阜": 126200,
    "愛知": 126201,
    "三重": 126202,
    "滋賀": 126203,
    "京都": 126204,
    "大阪": 126205,
    "兵庫": 126206,
    "奈良": 126207,
    "和歌山": 126208,
    "鳥取": 126209,
    "島根": 126210,
    "岡山": 126211,
    "広島": 126212,
    "山口": 126213,
    "徳島": 126214,
    "香川": 126215,
    "愛媛": 126216,
    "高知": 126217,
    "福岡": 126218,
    "佐賀": 126219,
    "長崎": 126220,
    "熊本": 126221,
    "大分": 126222,
    "宮崎": 126223,
    "鹿児島": 126224,
    "沖縄": 126225,
}

# 結果を保存するリスト
results = []

# データベースの設定
create_table_sql = """
CREATE TABLE IF NOT EXISTS hotels (
    hotel_name TEXT,
    price INTEGER,
    rating TEXT,
    highly_rated TEXT,
    pref TEXT  -- 都道府県名を保存するカラムを追加
)
"""

# データベースに接続
conn = sqlite3.connect("jalan_hotels.db")
c = conn.cursor()

# テーブルが存在する場合は削除
c.execute("DROP TABLE IF EXISTS hotels")
print("既存のテーブルを削除しました。")

# テーブルを作成
c.execute(create_table_sql)
print("新しいテーブルを作成しました。")

# 都道府県ごとにスクレイピング
for pref, ssc in pref_codes.items():
    print(f"{pref} のデータ取得を開始します...")
    
    # ページカウントとインデックスを都道府県ごとにリセット
    page_count = 0
    idx = 0
    pref_results = []  # 各都道府県の結果を保存するリスト

    while True:
        # URLを構築
        full_url = f"{base_url}{query_base}ssc={ssc}{query_mid}idx={idx}"
        print(f"取得中: {pref}: {full_url}")

        # ページを取得
        response = requests.get(full_url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # 宿泊施設データを取得
            items = soup.select('.p-yadoCassette__summary')  # 宿泊施設のリストを取得
            if not items:
                break  # データがない場合、ループを終了

            for item in items:
                hotel_name_tag = item.select_one('.p-searchResultItem__facilityName')
                price_tag = item.select_one('.p-searchResultItem__lowestPriceValue')
                rating_tag = item.select_one('.p-searchResultItem__summaryaverage-num')
                highly_rated_tag = item.select_one('.p-searchResultItem__highlyRated strong')

                hotel_name = hotel_name_tag.text.strip() if hotel_name_tag else ""
                
                price = price_tag.text.strip() if price_tag else ""
                price = re.sub(r'[^\d]', '', price)  # 数字以外の文字を削除

                # 数値が存在する場合は 1/2 の値を取得
                price = int(price) // 2 if price else 0


                rating = rating_tag.text.strip() if rating_tag else ""
                highly_rated = highly_rated_tag.text.strip() if highly_rated_tag else ""

                # 都道府県名を追加
                pref_results.append({
                    "hotel_name": hotel_name,
                    "price": price if price else None,  # priceがない場合はNone
                    "rating": rating,
                    "highly_rated": highly_rated,
                    "pref": pref  # 都道府県名を追加
                })

            # 次のインデックスを更新
            idx += 30  # 1ページ30件

            # データ数が少ない場合はループを抜ける（ページの最終ページに到達）
            if len(items) < 30:
                break

            page_count += 1
            print(f"ページ {page_count} を取得しました。")
            time.sleep(1)  # サーバー負荷軽減のため1秒待機
        else:
            print(f"{pref} のページ {page_count + 1} の取得に失敗しました。")
            break
    
    # 各都道府県ごとのデータ件数を表示
    print(f"{pref} のデータ取得完了: {len(pref_results)} 件")

    # データを全体の結果に追加
    results.extend(pref_results)

# データベースに保存
insert_sql = "INSERT INTO hotels (hotel_name, price, rating, highly_rated, pref) VALUES (?, ?, ?, ?, ?)"
for result in results:
    c.execute(insert_sql, (result['hotel_name'], result['price'], result['rating'], result['highly_rated'], result['pref']))

# コミットしてクローズ
conn.commit()
conn.close()
print("新しいデータをデータベースに保存しました。")


既存のテーブルを削除しました。
新しいテーブルを作成しました。
北海道 のデータ取得を開始します...
取得中: 北海道: https://www.jalan.net/theme/ski/?roomCount=1&ssc=126179&adultNum=2&distCd=05&stayYear=&stayMonth=&dateUndecided=1&roomCrack=200000&vosFlg=6&mvTabFlg=1&stayDay=&screenId=UWW1402&activeSort=0&stayCount=1&rootCd=01126195&idx=0
ページ 1 を取得しました。
取得中: 北海道: https://www.jalan.net/theme/ski/?roomCount=1&ssc=126179&adultNum=2&distCd=05&stayYear=&stayMonth=&dateUndecided=1&roomCrack=200000&vosFlg=6&mvTabFlg=1&stayDay=&screenId=UWW1402&activeSort=0&stayCount=1&rootCd=01126195&idx=30
ページ 2 を取得しました。
取得中: 北海道: https://www.jalan.net/theme/ski/?roomCount=1&ssc=126179&adultNum=2&distCd=05&stayYear=&stayMonth=&dateUndecided=1&roomCrack=200000&vosFlg=6&mvTabFlg=1&stayDay=&screenId=UWW1402&activeSort=0&stayCount=1&rootCd=01126195&idx=60
北海道 のデータ取得完了: 83 件
青森 のデータ取得を開始します...
取得中: 青森: https://www.jalan.net/theme/ski/?roomCount=1&ssc=126180&adultNum=2&distCd=05&stayYear=&stayMonth=&dateUndecided=1&roomCrack=200000&vosFlg=6&mvTabFlg=1&stayDay=&screenI

# 分析

## 1.価格に基づく分析

## 2.評価に基づく分析

## 3.都道府県に基づく分析